## Population checks
Brief interactive notebook to check that starting populations are those intended.

In [ ]:
import pandas as pd
import plotly.express as px

from aust_covid.constants import set_project_base_path
from aust_covid.inputs import load_pop_data
from aust_covid.model import build_model
from emutools.tex import DummyTexDoc
from emutools.utils import load_param_info

project_paths = set_project_base_path("../")

In [ ]:
parameters = load_param_info()["value"].to_dict()
param_info = load_param_info()
dummy_doc = DummyTexDoc()
epi_model = build_model(dummy_doc, param_info["abbreviations"])
vacc_model = build_model(dummy_doc, param_info["abbreviations"], vacc_ext=True)

In [ ]:
init_pop = epi_model.get_initial_population(parameters=parameters)
model_pops = load_pop_data(dummy_doc)
for age in epi_model.stratifications["agegroup"].strata:
    for state in epi_model.stratifications["states"].strata:
        model_pops.loc[int(age), f"check_{state}"] = init_pop[init_pop.index.str.contains(f"states_{state}") & init_pop.index.str.contains(f"agegroup_{age}X")].sum()
model_pops.plot().update_layout(xaxis_title="population", yaxis_title="age group", title="Population by age and spatial patch, base analysis")

In [ ]:
vacc_init_pop = vacc_model.get_initial_population(parameters=parameters)
for age in vacc_model.stratifications["agegroup"].strata:
    for state in vacc_model.stratifications["states"].strata:
        model_pops.loc[int(age), f"check_{state}"] = vacc_init_pop[vacc_init_pop.index.str.contains(f"states_{state}") & vacc_init_pop.index.str.contains(f"agegroup_{age}X")].sum()
model_pops.plot().update_layout(xaxis_title="population", yaxis_title="age group", title="Population by age and spatial patch, vacc analysis")

In [ ]:
px.bar(model_pops.iloc[:, :2])

In [ ]:
imm_check = {imm: init_pop[init_pop.index.str.contains(f"immunity_{imm}")].sum() for imm in epi_model.stratifications["immunity"].strata}
imm_props = {k: round(v, 3) for k, v in imm_check.items()}
print(f"Proportions by immune status, base analysis: { {k: round(v / sum(imm_props.values()), 3) for k, v in imm_props.items()}}")
px.bar(pd.Series(imm_props)).update_layout(xaxis_title="", yaxis_title="", showlegend=False)

In [ ]:
vacc_init_pop = vacc_model.get_initial_population(parameters=parameters)
vacc_imm_check = {imm: vacc_init_pop[vacc_init_pop.index.str.contains(f"immunity_{imm}")].sum() for imm in vacc_model.stratifications["immunity"].strata}
vacc_imm_props = {k: round(v, 3) for k, v in vacc_imm_check.items()}
print(f"Proportions by immune status, vacc analysis: { {k: round(v / sum(vacc_imm_check.values()), 3) for k, v in vacc_imm_check.items()}}")
px.bar(pd.Series(vacc_imm_props)).update_layout(xaxis_title="", yaxis_title="", showlegend=False)